## Assignment 5

Assigned: 13 November 2018  
Due: 4 December 2018

In this assignment you will design a database and write functions to query it. I have provided data for 999 books from Tar Heel Reader. The books are provided in JSON format which is easy to read in Python. 

In [81]:
# setup
import sqlite3
import comp521

check, report = comp521.start('A5')

import json
books = json.load(open('books.json'))

Now the variable `books` is an array of dictionaries. A typical book looks like this:

     {u'categories': [u'Animals and Nature'],
     u'created': u'2015-04-02 07:57:35',
     u'email': u'cwhitei0@miitbeian.gov.cn',
     u'first_name': u'Christopher',
     u'language': u'es',
     u'last_name': u'White',
     u'login': u'cwhitei0',
     u'pages': [{u'height': 310,
       u'caption': u'La rosa m\xe1s hermosa',
       u'url': u'/cache/images/48/3329876048_2b480e2243.jpg',
       u'width': 500},
      {u'height': 375,
       u'caption': u'Aunque crezca en un desierto',
       u'url': u'/cache/images/19/5886437819_01932b8f38.jpg',
       u'width': 500},
      {u'height': 500,
       u'caption': u'siempre tiene donde estar',
       u'url': u'/cache/images/17/5886448317_fcbf6c7d4d.jpg',
       u'width': 375}],
     u'title': u'La rosa'}
 
`categories` is a list of strings each indicating a library-like classification for the book. The list may be empty. Assume that additional categories may be created in the future.

`email`, `first_name`, `last_name`, and `login` relate to the author of the book. Of course, one author may write many books and the information must remain consistent. `login` is unique for every author.

`language` is the two-letter code for the language of the book. 

`title` is, of course, the title of the book and certainly not unique.

`pages` is an array. For each page of the book we have several fields. `caption` is the caption or text for this page of the book. `url` is the URL for the picture and its dimensions are in `width` and `height`. The dimensions of a particular image are always the same. There is significant reuse of pictures across books. The `url` for a picture sometimes changes and must be changed across all instances.

## Your mission

Your mission is to design a database in BCNF, write code to load it from the `books` array, and code to query it in various ways as described below.

Imagine you are implemeting the database part of the backend for Tar Heel Reader.

In [82]:
# I'll create an in-memory database for you. Do all your work in here
import sqlite3
db = sqlite3.Connection(':memory:')
cursor = db.cursor()

## 1. Design the Database

Create a set of tables with schemas that are in Boyce-Codd Normal Form (_the values are determined by the key, the whole key, and nothing but the key_) or as close to it as you can reasonably get. Use the same names for attributes as in the JSON format data. Give any new attributes you create reasonably descriptive names.

I'm going to assume below that you can find a given book by a unique integer id that you (or the DB) will assign.

In [83]:
# Write code here to create the tables
# use the db variable from above

cursor = db.execute('DROP TABLE IF EXISTS "Categories"')
cursor = db.execute('CREATE TABLE Categories(bookid int, categories varchar(999))')
cursor = db.execute('DROP TABLE IF EXISTS "Books"')
cursor = db.execute('CREATE TABLE Books(bookid int, title string, login string, language string, created varchar(999))')
cursor = db.execute('DROP TABLE IF EXISTS "Authors"')
cursor = db.execute('CREATE TABLE Authors(login string, email varchar(999), firstname string, lastname string)')
cursor = db.execute('DROP TABLE IF EXISTS "Pages"')
cursor = db.execute('CREATE TABLE Pages(bookid int, pageid int, caption varchar(999))')
cursor = db.execute('DROP TABLE IF EXISTS "ImageInPage"')
cursor = db.execute('CREATE TABLE ImageInPage(bookid int, imageid int, pageid int)')
cursor = db.execute('DROP TABLE IF EXISTS "Images"')
cursor = db.execute('CREATE TABLE Images(imageid int, width int, height int, url varchar(999))')

## 2. Load the data into your database. 

You should not need additional datastructures besides the db itself to accomplish this. 

In [84]:
# Write your code here, use the same db as above
# the source data is already in the variable books
# you'll, of course, need to iterate over it.
bookid = 0
imageid = 0
for x in books:
    for y in x['categories']:
        cursor = db.execute('INSERT INTO Categories(bookid, categories) VALUES (?,?)', (bookid, y))
            
    cursor = db.execute('INSERT INTO Books(bookid, title, login, language, created) VALUES (?,?,?,?,?)', (bookid, x['title'], x['login'], x['language'], x['created']))
    
    cursor = db.execute('Select login FROM Authors A WHERE login = ?',(x['login'],))
    b = cursor.fetchone()    
    if b == None:
        cursor = db.execute('INSERT INTO Authors(login, email, firstname, lastname) VALUES (?,?,?,?)', (x['login'], x['email'], x['first_name'], x['last_name']))
    pageid = 0
    for c in x['pages']:
        
        cursor = db.execute('INSERT INTO Pages(bookid, pageid, caption) VALUES (?,?,?)',(bookid, pageid, c['caption']))
        
    
        cursor = db.execute('Select url FROM Images WHERE url = ?',(c['url'],))
        b = cursor.fetchone()    
        if b == None: 
            cursor = db.execute('INSERT INTO Images(imageid, width, height, url) VALUES (?,?,?,?)', (imageid, c['width'], c['height'], c['url']))
            
            imageid += 1
        newImg = db.execute('Select imageid FROM Images WHERE url = ?', (c['url'],)).fetchone()[0]
        cursor = db.execute('INSERT INTO ImageInPage(bookid, imageid, pageid) VALUES (?,?,?)', (bookid, newImg, pageid))
        pageid += 1 
    bookid += 1

### Verify your tables

I'm going to include some code below to help you verify the sizes of your tables. I _think_ that if you've factored and loaded them up as expected they should have the same number of entiries as mine.

In [85]:
# a helper for the questions below
def getTableSize(name):
    return cursor.execute('''select count(*) from %s''' % name).fetchone()[0]

# How many entries are in your authors table?
NumberOfAuthors = cursor.execute('''Select Count(login) From Authors''').fetchone()[0]
check("Part 2 Number Of Authors", NumberOfAuthors, points=5)
# How many images are in your table?
NumberOfImages = cursor.execute('''Select Count(url) From Images''').fetchone()[0]
check("Part 2 Number Of Images", NumberOfImages, points=5)
# How many pages?
NumberOfPages = cursor.execute('''Select Count(pageid) From Pages''').fetchone()[0]
check("Part 2 Number of Pages", NumberOfPages, points=5)

Part 2 Number Of Authors appears correct
Part 2 Number Of Images appears correct
Part 2 Number of Pages appears correct


## 3. List the categories with the number of books in each for a given language.

A visitor to Tar Heel Reader might want to see how many books in their native language are available in each category.

Fill in the body of the function so that it uses a single query to produce a list of tuples with the category first and the number of books in that category second. Sort by category in ascending order.

For my table design I was able to do this with a single SQL statement. 

Always use the ? parameter substitution form to avoid SQL injection attacks.

In [86]:
def listCategoryCounts(language):
    #Write your code here
    cursor = db.execute('''Select C.categories, Count(C.bookid) From Categories C, Books B WHERE language = ? AND B.language = language AND B.bookid = C.bookid group by C.categories order by C.categories''',(language,))
    return cursor.fetchall()

r = listCategoryCounts('en')

check('Part 3', r, points=20)

r

Part 3 appears correct


[('Alphabet', 64),
 ('Animals and Nature', 257),
 ('Art and Music', 41),
 ('Biographies', 28),
 ('Fairy and Folk Tales', 24),
 ('Fiction', 187),
 ('Foods', 58),
 ('Health', 22),
 ('History', 43),
 ('Holidays', 55),
 ('Math and Science', 80),
 ('Nursery Rhymes', 2),
 ('People and Places', 243),
 ('Poetry', 25),
 ('Recreation and Leisure', 111),
 ('Sports', 48)]

## 4. List the books in a category.

The next step for our visitor is to select a category and browse the available books. We don't want to send __all__ the books, that would take too long, so we will send only 12 books (a chunk) at a time.

Fill in the body of the function below so that it returns a list of tuples with the book's unique id, title, author's last name, page count, and the URL of the picture on the first page of the book. The frontend will use this information to construct a grid of book previews with a link to read each book.The books should be ordered with the most recently created books first.

You should do this with a single query.

Your result should look like this:

    [(u'Farm Animals', u'Murphy', 980, 8, u'/cache/images/89/291005289_a05ae73053.jpg'),
     ...
    ]


In [87]:
def listBooks(language, category, chunk):
    # language is the 2-letter language code
    # category is a category name
    # chunk is an integer. 0 means the first 12 books, 1 means the next 12, etc.
    # write your code here
    cursor = db.execute('''SELECT B.bookid, B.title, A.lastname, Count(DISTINCT P.pageid), I.url
                           FROM Pages P, Books B, Authors A, Images I, ImageInPage IP, Categories C 
                           WHERE C.bookid = B.bookid AND A.login = B.login AND B.bookid = P.bookid 
                           AND IP.bookid = P.bookid AND IP.imageid = I.imageid AND IP.pageid = 0
                           AND C.categories = ? AND B.language = ? 
                           Group By B.bookid 
                           Order By B.created DESC
                           limit ?, 12''',(category, language, chunk * 12))
    result = cursor.fetchall()
    return result
r1 = listBooks('en', 'Animals and Nature', 0)
# I'm stripping off the ID before testing
check('Part 4 page 1', [ r[1:] for r in r1 ], points=10)

r2 = listBooks('en', 'Animals and Nature', 1)
toRead = r2[-1][0] # get the id of the last book on the second page, we'll read it below
# I'm stripping off the ID before testing
check('Part 4 page 2', [ r[1:] for r in r2], points=10)

r1

Part 4 page 1 appears correct
Part 4 page 2 appears correct


[(998,
  'Seaside Aquarium',
  'Simpson',
  8,
  '/cache/images/21/122430121_a653d2f8ab.jpg'),
 (997,
  'The Funny Crab',
  'Bennett',
  5,
  '/cache/images/49/5638860949_d89877e19d.jpg'),
 (993,
  'Polar Bears',
  'Rogers',
  10,
  '/cache/images/70/3582475670_df8e8e8974.jpg'),
 (992, 'Horse', 'Rice', 6, '/cache/images/79/57632579_c7482974fe.jpg'),
 (989, 'The Mouse', 'Kelly', 13, '/cache/images/39/8093539_473996654f.jpg'),
 (935, 'Cats', 'Patterson', 8, '/cache/images/70/5110834170_0797f39278.jpg'),
 (986,
  'The Tired Lion',
  'Bennett',
  4,
  '/cache/images/57/5657669257_d4be2a3f98.jpg'),
 (976,
  'My book on how to help the Earth.',
  'Harper',
  8,
  '/cache/images/48/2435883148_0eba86944f.jpg'),
 (975,
  'Help the Earth',
  'Harper',
  7,
  '/cache/images/77/3717759677_4a520a1dbb.jpg'),
 (974,
  'My book about how to save the Earth',
  'Harper',
  8,
  '/cache/images/77/3717759677_4a520a1dbb.jpg'),
 (985,
  'Farm Animals',
  'Reynolds',
  8,
  '/cache/images/89/291005289_a05ae7

## 5. Fetch a single book

Now that our vistor has clicked on a book, we want to fetch all of its content so the Javascript in the browser can render it without further reference to the server. We want to produce a result formatted like the ones we read in earlier so create a Python dictionary and assign values to keys with the same names. Exclude the `id` from the result to make testing easier. Finally return a JSON encoded string, I've included that code for you inside the function. 

I can't see any way to do this step with only one query. I got it down to three. That isn't so bad because after this step our visitor will be happily reading a book without bothering us again for a while.

Fill in the body of the function below so that it produces a JSON encoded string that looks like one of the entries in your original books array. Do __not__ simply return an item from that array, you __must__ reconstruct the book from the data in your database.

In [88]:
def fetchBook(id):
    
    #Write your code here
    bookTemp = db.execute('''SELECT * FROM Books B WHERE B.bookid = ?''',(id,))
    categoriesTemp = db.execute('''SELECT C.categories FROM Categories C WHERE C.bookid = ?''',(id,))
    authorsTemp = db.execute('''SELECT A.login, A.email, A.firstname, A.lastname FROM Authors A, Books B
                           WHERE B.bookid = ? AND A.login = B.login''',(id,))
    pageTemp = db.execute('''SELECT Distinct I.url, P.caption, I.height, I.width 
                            FROM Pages P, Images I, ImageInPage IP 
                            WHERE P.bookid = IP.bookid AND P.pageid = IP.pageid AND I.imageid = IP.imageid AND P.bookid = ? 
                            ''',(id,))
    
    page = pageTemp.fetchall()
    book = bookTemp.fetchone()
    author = authorsTemp.fetchone()
    categories = categoriesTemp.fetchone()
    #INSERT INTO Books(bookid, title, login, language, created)
    pages = [{"caption": r[1], "height": r[2], "url": r[0], "width": r[3]} for r in page]
    book = {"categories": [categories[0]], "created": book[4], "email": author[1], "first_name": author[2], 
               "language": book[3],"last_name": author[3],"login": author[0],"pages": pages, "title": book[1]}
    
    print(book)
    return json.dumps(book, sort_keys=True)
b = fetchBook(toRead)
check('Part 5', b, points=30)


{'categories': ['Animals and Nature'], 'created': '2015-03-13 11:49:30', 'email': 'eleeim@bloomberg.com', 'first_name': 'Ernest', 'language': 'en', 'last_name': 'Lee', 'login': 'eleeim', 'pages': [{'caption': 'Sea turtles belong to the same family as land turtles.', 'height': 333, 'url': '/cache/images/14/2341383314_5347b601a7.jpg', 'width': 500}, {'caption': "Sea turtles' front legs look like flippers. They steer with their back legs.", 'height': 375, 'url': '/cache/images/03/10449014403_c8d853c373.jpg', 'width': 500}, {'caption': 'Sea Turtles are cold blooded.', 'height': 375, 'url': '/cache/images/88/3706977488_08208d18db.jpg', 'width': 500}, {'caption': 'Sea Turtles eat plants and can go for a year without eating.', 'height': 375, 'url': '/cache/images/05/4018269705_b8f4c692ee.jpg', 'width': 500}, {'caption': 'Sea Turtles need to breathe air to live.', 'height': 375, 'url': '/cache/images/16/5787913916_6b7af6eba2.jpg', 'width': 500}, {'caption': 'Sea Turtles can hold their breath f

In [89]:
def listAuthor(login):
    # login is the unique login for an author
    cursor = db.execute('''SELECT B.bookid, B.title, A.lastname, Count(DISTINCT P.pageid), I.url
                           FROM Pages P, Books B, Authors A, Images I, ImageInPage IP, Categories C 
                           WHERE C.bookid = B.bookid AND A.login = B.login AND B.bookid = P.bookid 
                           AND IP.bookid = P.bookid AND IP.imageid = I.imageid AND IP.pageid = 0
                           AND A.login = ? 
                           Group By B.bookid 
                           Order By B.created DESC''',(login,))
    result = cursor.fetchall()
    return result

r1 = listAuthor('jhansenrr')
# I'm stripping off the ID before testing
check('Part 6', [ r[1:] for r in r1 ], points=15)

r1

Part 6 appears correct


[(0,
  'PLURALS: cuniculus ursum amat.',
  'Hansen',
  8,
  '/cache/images/06/357393306_bb325a5c92.jpg'),
 (2,
  'Plurals: cuniculus et amicus',
  'Hansen',
  8,
  '/cache/images/67/219674667_b3aeb1528e.jpg'),
 (1,
  'Plurals: Canes, nomine Montius et Daisia',
  'Hansen',
  9,
  '/uploads/2009/07/annapmagistra-1246609542.jpg'),
 (14,
  'Cuniculus versipellem salutat',
  'Hansen',
  14,
  '/cache/images/39/3178513639_d71ae707ab.jpg')]

## Done!

Get your report, save, and upload your notebook on Sakai.

In [92]:
onyen = 'fiji'
collaborators = []
report(onyen, collaborators)

  Collaborators: []
Report for fiji
  8 of 8 appear correct, 100 of 100 points
